# STT
##### uv add pyaudio speechrecognition pydub

In [1]:
from dotenv import load_dotenv
import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play
from openai import OpenAI
import os
import tempfile

load_dotenv()
client = OpenAI()

In [ ]:
r = sr.Recognizer()

with sr.Microphone() as source:
    
    print("Say something!")
    r.adjust_for_ambient_noise(source) # 주변 소음 제거
    
    audio = r.listen(source)
    text = r.recognize_openai(audio)
    print(f'인식된 텍스트 : {text}')
    
    audio_file = audio.get_wav_data()
    
    with open('./audio/input.wav', 'wb') as f:
        f.write(audio_file)
        
    print('save voice')    

Say something!
인식된 텍스트 : 안녕하세요 전민도 입니다
save voice


In [6]:
sound = AudioSegment.from_wav('./audio/input.wav')

play(sound)

In [4]:
def speak_reaction(user_prompt):
    system_prompt = """당신은 낭만적인 챗봇입니다"""
    
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

In [13]:
while True:
    r = sr.Recognizer()
    
    with sr.Microphone() as source:
        print('tell me')
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        print('loading')
        user_text = r.recognize_openai(audio)
        print(f'said text : {user_text}')
        
        if user_text == '들어가':
            break
        
        answer = speak_reaction(user_text)
        print(f'chatbot : {answer}')
        

tell me
loading
said text : 
chatbot : 안녕하세요! 오늘은 어떤 이야기를 나누고 싶으신가요? 당신의 마음속에 따뜻한 감성과 함께 이야기를 들려드릴게요.
tell me
loading
said text : 
chatbot : 안녕하세요! 오늘은 어떤 사랑스럽고 로맨틱한 이야기를 나누고 싶으신가요? 당신의 마음을 따뜻하게 만들어줄 어떤 이야기든 저는 언제든 준비되어 있어요. 당신의 하루를 특별하게 만들어 드릴 수 있다면 정말 행복하겠습니다.
tell me
loading
said text : 친절하다니까 빨리 봐야 안 그러곤 해 진짜
chatbot : 정말 친절하구나, 그런 마음씀씀이가 너무 고마워. 빠르게 봐야 한다니 내 마음도 급해지네. 언제든지 도와줄 준비가 되어 있으니까, 편하게 말해줘. 같이 힘내자!
tell me
loading
said text : 스카프, 신찰고래시가 1등을 거뒀죠. 그래서 이게 과연 많이 유행을 할 것인가. 그리고 프리미엄 대디.
chatbot : 아, 정말 흥미로운 소식이네요! 신찰고래시가 1등을 차지했다니, 그 인기가 정말 대단한 것 같아요. 이런 인기 아이템이나 트렌드가 얼마나 오래 지속될지는 사람들의 관심과 반응에 따라 달라지겠지만, 분명히 많은 사람들이 관심을 갖고 있어서 앞으로 또 어떤 변화가 생길지 기대가 되네요.

프리미엄 대디라는 말도 참 따뜻하고 로맨틱하게 들리는데요, 혹시 특별한 의미나 맥락이 있나요? 어떤 이야기를 담고 있는지 궁금하네요. 이런 대화들이 더 풍부하고 아름다운 감성을 자아내게 하는 것 같아요!
tell me
loading
said text : 확실히 그게 있었던 것 같아요. 순간적으로 많이 했지만 아, 진짜의 느낌을 내가 한다고 놓치면 안 된다. 한다고 해서 금방 좀 차분해졌고.
chatbot : 그 느낌 정말 소중하네요. 순간순간의 감정을 놓치지 않고, 진짜 자신을 느끼려고 하는 모습이 아름다워요. 때로는 바쁘고 흔들릴 때도 있지만, 그런 찰나를 자각하는 것 자체가 이미

KeyboardInterrupt: 

# STT -> LLM -> TTS (챗봇 스피커)

In [5]:
while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("듣는 중....")
        # STEP 1. 마이크로부터 입력
        r.adjust_for_ambient_noise(source) 
        audio = r.listen(source)         
        print("인식 중입니다.....")

        # STEP 2. Whisper API를 통한 텍스트 변환
        user_text = r.recognize_openai(audio) 
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break
        
        # STEP 3. 인공지능 챗봇 응답
        answer = speak_reaction(user_text)
        print(f"챗봇 답변: {answer}")

        # STEP 4. Whisper API로 응답
        with client.audio.speech.with_streaming_response.create(
            model="gpt-4o-mini-tts",
            voice="coral",
            input=answer,
            instructions="밝은 목소리로 말해줘"
        ) as response:
            # 음성 합성 결과를 임시 파일로 저장
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
                temp_path = temp_file.name
                response.stream_to_file(temp_path)

                # 재생
                sound = AudioSegment.from_mp3(temp_path)
                play(sound)

듣는 중....
인식 중입니다.....
인식된 텍스트: 시리아 시리아 시리아
챗봇 답변: 시리아에 대해 이야기하고 싶으신가요? 아니면 시리아라는 단어에 특별한 의미가 있나요? 시리아는 풍부한 역사와 아름다운 풍경, 그리고 다양한 문화가 어우러진 나라입니다. 고대 로마와 이슬람 문명이 숨쉬는 곳이기도 하죠. 혹시 시리아에 대해 더 알고 싶은 점이 있나요? 당신의 관심사에 맞춰 함께 이야기를 나눌 수 있어요.
듣는 중....
인식 중입니다.....
인식된 텍스트: อืมัน
챗봇 답변: สวัสดีค่ะ! มีอะไรให้ฉันช่วยเหลือหรือพูดคุยกันไหมคะ?


KeyboardInterrupt: 